Move into `ControlNet-XS` directory to run

Note: I made these changes to make the code run:
1. In `requirements/pt2.txt`: `torchaudio==2.0.2`,`torchvision==0.15.2`
2. In `ldm/models/diffusion/ddpm.py`: `from pytorch_lightning.utilities.rank_zero import rank_zero_only`

In [1]:
import scripts.control_utils as cu
import torch
from PIL import Image

Info: `UmerDebugLogger` created. This is a logging class that will be deleted when the PR to integrate ControlNet-XS is done.


/opt/conda/lib/python3.10/site-packages/torchvision/transforms/functional_tensor.py:5: UserWarning: The torchvision.transforms.functional_tensor module is deprecated in 0.15 and will be **removed in 0.17**. Please don't rely on it. You probably just need to use APIs in torchvision.transforms.functional or in torchvision.transforms.v2.functional.
  warnings.warn(


Downloading: "https://huggingface.co/lllyasviel/ControlNet/resolve/main/annotator/ckpts/dpt_hybrid-midas-501f0c75.pt" to /home/ControlNet-XS/annotator/ckpts/dpt_hybrid-midas-501f0c75.pt



100%|██████████| 470M/470M [00:24<00:00, 20.0MB/s] 
/opt/conda/lib/python3.10/site-packages/timm/models/_factory.py:117: UserWarning: Mapping deprecated model name vit_base_resnet50_384 to current vit_base_r50_s16_384.orig_in21k_ft_in1k.
  model = create_fn(


In [2]:
path_to_config = '../Heidelberg_model_configs/sdxl_encD_canny_48m.yaml'
model = cu.create_model(path_to_config).to('cuda')

Info: `UmerDebugLogger` created. This is a logging class that will be deleted when the PR to integrate ControlNet-XS is done.
Building a Downsample layer with 2 dims.
  --> settings are: 
 in-chn: 320, out-chn: 320, kernel-size: 3, stride: 2, padding: 1
constructing SpatialTransformer of depth 2 w/ 640 channels and 10 heads
constructing SpatialTransformer of depth 2 w/ 640 channels and 10 heads
Building a Downsample layer with 2 dims.
  --> settings are: 
 in-chn: 640, out-chn: 640, kernel-size: 3, stride: 2, padding: 1
constructing SpatialTransformer of depth 10 w/ 1280 channels and 20 heads
constructing SpatialTransformer of depth 10 w/ 1280 channels and 20 heads
constructing SpatialTransformer of depth 10 w/ 1280 channels and 20 heads
constructing SpatialTransformer of depth 10 w/ 1280 channels and 20 heads
constructing SpatialTransformer of depth 10 w/ 1280 channels and 20 heads
constructing SpatialTransformer of depth 10 w/ 1280 channels and 20 heads
constructing SpatialTransforme

Some weights of the model checkpoint at openai/clip-vit-large-patch14 were not used when initializing CLIPTextModel: ['vision_model.encoder.layers.20.self_attn.out_proj.bias', 'vision_model.encoder.layers.14.mlp.fc2.bias', 'vision_model.encoder.layers.11.self_attn.q_proj.weight', 'vision_model.encoder.layers.6.layer_norm1.weight', 'vision_model.encoder.layers.23.mlp.fc1.bias', 'vision_model.encoder.layers.16.mlp.fc2.bias', 'vision_model.encoder.layers.18.self_attn.k_proj.bias', 'vision_model.encoder.layers.10.self_attn.q_proj.bias', 'logit_scale', 'vision_model.encoder.layers.8.mlp.fc1.weight', 'vision_model.encoder.layers.9.self_attn.k_proj.weight', 'vision_model.encoder.layers.12.self_attn.v_proj.weight', 'vision_model.encoder.layers.6.layer_norm1.bias', 'vision_model.encoder.layers.7.layer_norm1.bias', 'vision_model.encoder.layers.18.self_attn.q_proj.weight', 'vision_model.encoder.layers.8.layer_norm1.bias', 'vision_model.encoder.layers.22.self_attn.q_proj.bias', 'vision_model.encod

Initialized embedder #0: FrozenCLIPEmbedder with 123060480 params. Trainable: False
Initialized embedder #1: FrozenOpenCLIPEmbedder2 with 694659841 params. Trainable: False
Initialized embedder #2: ConcatTimestepEmbedderND with 0 params. Trainable: False
Initialized embedder #3: ConcatTimestepEmbedderND with 0 params. Trainable: False
Initialized embedder #4: ConcatTimestepEmbedderND with 0 params. Trainable: False
making attention of type 'vanilla-xformers' with 512 in_channels
building MemoryEfficientAttnBlock with 512 in_channels...
Working with z of shape (1, 4, 32, 32) = 4096 dimensions.
making attention of type 'vanilla-xformers' with 512 in_channels
building MemoryEfficientAttnBlock with 512 in_channels...
Loaded model config from [../Heidelberg_model_configs/sdxl_encD_canny_48m.yaml]


In [3]:
canny_high_th = 250
canny_low_th = 100
size = 768
num_samples=2

image = cu.get_image('../input_images/shoe_cloud.png', size=size)
edges = cu.get_canny_edges(image, low_th=canny_low_th, high_th=canny_high_th)

In [4]:
edges

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [5]:
from sgm.umer_debug_logger import udl

udl.set_dir('../logs/cloud_xl/', clear=True)
udl.set_condition([udl.SUBBLOCK, udl.SUBBLOCKM1])
udl.load_input('../logs/input_cache_xl/', x='x.pt', t='t.pt', xcross='xcross.pt', hint='hint.pt', add_embeds='add_embeds.pt')
assert udl.input_action == 'load'

In [6]:
samples, controls = cu.get_sdxl_sample(
    guidance=edges,
    ddim_steps=10,
    num_samples=num_samples,
    model=model,
    shape=[4, size // 8, size // 8],
    control_scale=0.95,
    prompt='cinematic, advertising shot, shoe in a city street, photorealistic shoe, colourful, highly detailed',
    n_prompt='low quality, bad quality, sketches'
)

Global seed set to 1999158951


[CONTROL CORRECTION OF ControlledDiffusionEngine SCALED WITH 0.95]
crop_coords_top_left = tensor([[0, 0],
        [0, 0]], device='cuda:0')
original_size_as_tuple = tensor([[768, 768],
        [768, 768]], device='cuda:0')
target_size_as_tuple = tensor([[768, 768],
        [768, 768]], device='cuda:0')
These are the timesteps:
> 0: 14.614641189575195
> 1: 8.302803039550781
> 2: 5.08776330947876
> 3: 3.3210830688476562
> 4: 2.276463031768799
> 5: 1.6128861904144287
> 6: 1.1605783700942993
> 7: 0.8298594951629639
> 8: 0.5692849159240723
> 9: 0.3416738212108612
> 10: 0.0
control scale:  tensor([0.9500, 0.9500, 0.9500, 0.9500, 0.9500, 0.9500, 0.9500, 0.9500, 0.9500,
        0.9500, 0.9500, 0.9500, 0.9500, 0.9500, 0.9500, 0.9500, 0.9500, 0.9500,
        0.9500], device='cuda:0')
[udl] Input loaded (batch size = 1)
Of course I've not learned a time embedding. I'm smart! Let me collaborate with the base model by {self.control_scale:.2f}**3.
[udl] Input loaded (batch size = 1)


SystemExit: It is done, my dude. Let us look at these tensors. - 2024-01-17 22:14:37

/opt/conda/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3513: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
